## STARTING CODE

In [1]:
# IMPORT STANDART LIBRARIES
import os
import re
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import seaborn as sns
from IPython.display import display

import win32con, win32api, os

import warnings; warnings.simplefilter('ignore')
sns.set()

# IMPORT OWN FUNCTIONS
from save_data import save_csv, save_xlsx

# FILES:
# Root-folder path
main_path = 'Z:\\Projects\\РФФИ-Норвегия\\Данные\\Экспериментальные данные в лаборатории\\Droplet freezing facility data\\Generating droplets'
# Dataframe-name
dataframe_name = 'GENERATING_DATA.csv'
xlsx_name = 'GENERATING_DATA.xlsx'

# name of folder containing videos
video_folder = 'VIDEO_DATA'
# cropped-results folder
cropped_folder = 'cropping_results'
# Cropping-folder path
cropping_path = os.path.join(main_path, video_folder, cropped_folder)

# Name of folder containing droplet properties
imgproc_folder = 'IMAGE_PROCESSING_DATA'
imgproc_path = os.path.join(main_path, imgproc_folder)

# opening dataframes
df = pd.read_csv(os.path.join(main_path, dataframe_name))

# deleting "Unnamed columns"
#df = df.loc[:,~df.columns.str.match("Unnamed")]

In [13]:
# Show largest droplets
largest_drops_df = df[(df.pressure==1000)&(df.crop==0)&(df.needle=='N01-1')&(df.impulse_time==4000)&(df.droplet_size>0)].sort_values(['droplet_size'],ascending=False)[['test','crop','voltage','pressure','droplet_size','sigma_droplet_size','starting_velocity','comments']]

## Save results

In [2]:
# SAVE IN CSV 
save_name = dataframe_name
save_csv(df,main_path, save_name)
# SAVE IN XLSX 
save_name = xlsx_name
save_xlsx(df,main_path, xlsx_name)

Are you sure you want to write this DataFrame to main DataBase? Print 'yes'
2021-11-02#1338 GENERATING_DATA.csv
Are you sure you want to write this DataFrame to Excel-file? Print 'yes'


'Done'

## Add columns from image_processing.csv

#### Modify image_processing data before merging

In [2]:
# Create calibration df
calibrat_df = df[df['test_type']!='calibrat'][['test','pixel_size','record_rate']]
# calibrat_df.info()
# Read image processing df
im_proc_df = pd.read_csv(os.path.join(main_path,'IMAGE_PROCESSING_DATA','image_processing.csv'))
# im_proc_df.info()
# Merge improc and calibr dataframes
im_proc_df = im_proc_df.merge(calibrat_df, how='left',on='test')
# Getting new columns
im_proc_df['droplet_count'] = round(im_proc_df['autoDropCount'])
im_proc_df['flow_type'] = im_proc_df['autoFlowType'].str.lower()
im_proc_df['droplet_size'] = round(im_proc_df['pixDiameter']*im_proc_df['pixel_size'],2)
im_proc_df['sigma_droplet_size'] = round(im_proc_df['sigmaPixDiameter']*im_proc_df['pixel_size'],2)
im_proc_df['starting_velocity'] = round(im_proc_df['startingPixVelocity']*im_proc_df['pixel_size']*1e-6*im_proc_df['record_rate'],5)
im_proc_df['ending_velocity'] = round(im_proc_df['endingPixVelocity']*im_proc_df['pixel_size']*1e-6*im_proc_df['record_rate'],5)
display(im_proc_df.info())
# take necessary part of image processing df
part_im_proc_df = im_proc_df[['test', 'crop', 'droplet_count', 'flow_type', 'droplet_size', 'sigma_droplet_size', 'starting_velocity', 'ending_velocity']]
display(part_im_proc_df.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471 entries, 0 to 470
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   test                 471 non-null    int64  
 1   crop                 471 non-null    int64  
 2   autoFlowType         471 non-null    object 
 3   autoDropCount        471 non-null    int64  
 4   pixDiameter          471 non-null    float64
 5   sigmaPixDiameter     471 non-null    float64
 6   startingPixVelocity  471 non-null    float64
 7   endingPixVelocity    471 non-null    float64
 8   pixel_size           471 non-null    float64
 9   record_rate          471 non-null    float64
 10  droplet_count        471 non-null    int64  
 11  flow_type            471 non-null    object 
 12  droplet_size         471 non-null    float64
 13  sigma_droplet_size   471 non-null    float64
 14  starting_velocity    471 non-null    float64
 15  ending_velocity      471 non-null    flo

None

,test,crop,droplet_count,flow_type,droplet_size,sigma_droplet_size,starting_velocity,ending_velocity
0,29,5,1,single,1066.67,9.98,0.13734,0.70394
1,29,4,1,single,1084.20,12.11,0.02767,0.70474
2,29,3,1,single,1081.13,10.12,0.07925,0.70379
3,29,2,1,single,1078.71,16.33,0.12760,0.70719
4,29,1,1,single,1077.44,19.23,0.11012,0.71214


#### Merge image processing results with generating data

In [ ]:
merge_df = df.merge(part_im_proc_df, how='outer',on=['test','crop'])
# Filling of crops-NaN parameters (pressure, voltage, etc)
full_df = pd.DataFrame(columns=merge_df.columns)
for testNum in df['test'].unique():
    full_df = full_df.append(merge_df[merge_df['test']==testNum].fillna(axis=0,method='ffill'),ignore_index=True)
full_df.sort_values(by=['test','crop'],inplace=True)
full_df.info()

# SAVE IN CSV 
save_name = dataframe_name
save_csv(full_df,main_path, save_name)
# SAVE IN XLSX 
save_name = xlsx_name
save_xlsx(full_df,main_path, xlsx_name)

## Get analyzed columns from imgproc_detailed.csv and get median values
**imgproc_detailed.csv** recieved after using **imgproc_analyzer.ipynb**

#### Get characteristics from improc_df (detailed)

In [3]:
improc_df = pd.read_csv(os.path.join(imgproc_path,'0029-0277_imgproc_detailed.csv'))
improc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8555 entries, 0 to 8554
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   test          8555 non-null   float64
 1   crop          8555 non-null   float64
 2   frameNum      8555 non-null   float64
 3   y_pix         8555 non-null   float64
 4   x_pix         8555 non-null   float64
 5   diameter_pix  8555 non-null   float64
 6   area_pix      8555 non-null   float64
 7   pixel_size    8555 non-null   float64
 8   record_rate   8555 non-null   float64
 9   time          8555 non-null   float64
 10  y             8555 non-null   float64
 11  x             8555 non-null   float64
 12  del_y         8555 non-null   float64
 13  del_x         8555 non-null   float64
dtypes: float64(14)
memory usage: 935.8 KB


In [4]:
# function of getting characteristics from improc_df
'''
    fill_df = pd.DataFrame(columns = df.columns)
    fill_df['fall_time'] = None
    fill_df['vertical_path'] = None
    fill_df['max_horizontal_deviation'] = None
'''
df['fall_time'] = None
df['vertical_path'] = None
df['max_horizontal_deviation'] = None

def get_improc_params(row):
    # analyze only Single-droplets rows
    if row['droplet_count'] != 1:
        return row
    # time in seconds
    row['fall_time'] = improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['time'].max()
    # path and deviation in mm
    row['vertical_path'] = improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['del_y'].max()
    row['max_horizontal_deviation'] = improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['del_x'].apply(abs).max()   
    return row

df = df.apply(get_improc_params, axis=1)
display(df[df['droplet_count']==1][['droplet_size','sigma_droplet_size',
'starting_velocity','ending_velocity','fall_time','vertical_path', 'max_horizontal_deviation']].describe())

,droplet_size,sigma_droplet_size,starting_velocity,ending_velocity,fall_time,vertical_path,max_horizontal_deviation
count,229.000000,229.000000,229.000000,229.000000,157.000000,157.000000,157.000000
mean,983.809410,21.320611,0.203545,0.699686,0.054118,24.614656,0.516654
std,367.646499,14.754176,0.098630,0.087537,0.007519,2.430278,0.366752
min,0.000000,0.000000,-0.012510,0.000000,0.020000,17.833681,0.010343
25%,772.600000,9.430000,0.167850,0.688370,0.051000,22.610478,0.252259
50%,947.010000,17.610000,0.204490,0.718970,0.055000,25.743053,0.381990
75%,1046.455000,29.195000,0.241680,0.734510,0.058000,26.456063,0.751766
max,2097.310000,76.820000,0.995950,1.118580,0.075000,27.620478,1.461842


#### Get median values for "crop 0" of single droplets

In [5]:
def get_median_crop_params(row):
    if row['crop'] != 0:
        return row
    # get drop count
    mean_value = df[(df['test']==row['test'])&(improc_df['crop']!=row['crop'])]['droplet_count'].mean()
    if ~np.isnan(mean_value):
        if mean_value == 0:
            row['droplet_count'] = 0
        elif mean_value < 2:
            row['droplet_count'] = 1
        else:
            row['droplet_count'] = math.floor(mean_value)

    # add flow_type
    if row['droplet_count'] == 0:
        row['flow_type'] = 'no'
    elif row['droplet_count'] == 1:
        row['flow_type'] = 'single'
    elif row['droplet_count'] <= 4:
        row['flow_type'] = 'multi'
    elif row['droplet_count'] > 4:
        row['flow_type'] = 'jet'

    # get median characteristics
    if row['droplet_count'] == 1:
        values = df[(df['test']==row['test'])&(improc_df['crop']!=row['crop'])][['droplet_size','sigma_droplet_size','starting_velocity','ending_velocity', 'fall_time','vertical_path','max_horizontal_deviation']]
        if df[(df['test']==row['test'])&(improc_df['crop']!=row['crop'])]['droplet_size'].median() > 0:
            row[['droplet_size','sigma_droplet_size','starting_velocity','ending_velocity', 'fall_time','vertical_path','max_horizontal_deviation']] = values.mean()
        else:
            row[['droplet_size','sigma_droplet_size','starting_velocity','ending_velocity', 'fall_time','vertical_path','max_horizontal_deviation']] = values.max()

        
    return row

df = df.apply(get_median_crop_params, axis=1)


In [6]:
# SAVE IN CSV 
save_name = dataframe_name
save_csv(df,main_path, save_name)
# SAVE IN XLSX 
save_name = xlsx_name
save_xlsx(df,main_path, xlsx_name)

Are you sure you want to write this DataFrame to main DataBase? Print 'yes'
2021-11-03#1331 GENERATING_DATA.csv
Are you sure you want to write this DataFrame to Excel-file? Print 'yes'


'Done'